<a href="https://colab.research.google.com/github/strange-hawk/tensorflow_examples/blob/main/deep_CNN_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras as keras
from tensorflow.keras import datasets, layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
EPOCHS=50 
NUM_CLASSES = 10

In [3]:
def load_data():
  (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
  x_train = np.float32(x_train)
  x_test = np.float32(x_test)
  mean = np.mean(x_train,axis=(0,1,2,3)) 
  std = np.std(x_train,axis=(0,1,2,3)) 
  x_train = (x_train-mean)/(std+1e-7) 
  x_test = (x_test-mean)/(std+1e-7)

  y_train = tf.keras.utils.to_categorical(y_train,NUM_CLASSES) 
  y_test = tf.keras.utils.to_categorical(y_test,NUM_CLASSES)

  return x_train, y_train, x_test, y_test

In [4]:
(x_train, y_train, x_test, y_test) = load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [5]:
def build_model():
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3,3), padding='same', input_shape=x_train.shape[1:], activation='relu')) 
  model.add(layers.BatchNormalization()) 
  model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu')) 
  model.add(layers.BatchNormalization()) 
  model.add(layers.MaxPooling2D(pool_size=(2,2))) 
  model.add(layers.Dropout(0.2))

  model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
  model.add(layers.BatchNormalization()) 
  model.add(layers.MaxPooling2D(pool_size=(2,2))) 
  model.add(layers.Dropout(0.3))

  model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
  model.add(layers.BatchNormalization()) 
  model.add(layers.MaxPooling2D(pool_size=(2,2))) 
  model.add(layers.Dropout(0.4))

  model.add(layers.Flatten()) 
  model.add(layers.Dense(10, activation='softmax')) 
  
  return model


In [6]:
model = build_model()

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        1

In [8]:
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

In [9]:
model.fit(x_train, y_train, batch_size=64, epochs=EPOCHS, validation_data=(x_test,y_test))

Epoch 1/50
782/782 [==============================] - 7s 10ms/step - loss: 1.7301 - accuracy: 0.4804 - val_loss: 1.3055 - val_accuracy: 0.5910
Epoch 2/50
782/782 [==============================] - 7s 9ms/step - loss: 1.0569 - accuracy: 0.6531 - val_loss: 0.8472 - val_accuracy: 0.7166
Epoch 3/50
782/782 [==============================] - 7s 9ms/step - loss: 0.8348 - accuracy: 0.7200 - val_loss: 0.8780 - val_accuracy: 0.7104
Epoch 4/50
782/782 [==============================] - 7s 9ms/step - loss: 0.7176 - accuracy: 0.7546 - val_loss: 0.6815 - val_accuracy: 0.7701
Epoch 5/50
782/782 [==============================] - 7s 9ms/step - loss: 0.6404 - accuracy: 0.7814 - val_loss: 0.6903 - val_accuracy: 0.7716
Epoch 6/50
782/782 [==============================] - 7s 9ms/step - loss: 0.5833 - accuracy: 0.8009 - val_loss: 0.6514 - val_accuracy: 0.7812
Epoch 7/50
782/782 [==============================] - 7s 9ms/step - loss: 0.5360 - accuracy: 0.8161 - val_loss: 0.5612 - val_accuracy: 0.8125
Epoch

In [10]:
model.evaluate(x_test, y_test, batch_size=64)

157/157 [==============================] - 1s 4ms/step - loss: 0.5093 - accuracy: 0.8638


[0.5092526078224182, 0.8637999892234802]

In [11]:
datagen = ImageDataGenerator(rotation_range=30, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True, zoom_range=0.2)
datagen.fit(x_train)

In [ ]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=64), epochs=EPOCHS, verbose=1,validation_data=(x_test,y_test))

In [15]:
model_json = model.to_json()

In [16]:
with open('model.json','w') as json_file:
  json_file.write(model_json)

In [17]:
model.save_weights('model.h5')